To launch interactive shell in Slurm:
```
srun -c 16 --gres=gpu:4 --mem=96G --time=45:00:00 --pty bash -i

srun -c 4 --gres=gpu:1 --mem=36G --time=1:00:00 --pty bash -i
```

In [ ]:
import torch
from torchvision import models

model = models.mobilenet_v2()

for param in model.namedparameters():
    print(type(param))
    break

## Prepare data

This is an artifact of the implementation of the datamodule, the cached file will have synchronization problem when launch multiple training instances. So we prepare the subset once here. 
TODO: fix the synchronization problem.

In [ ]:
from datamodules import cifar100_datamodule
from pathlib import Path

training_labels = (30, 67, 62, 10, 51, 22, 20, 24, 97, 76)
training_labels_2 = (55, 91, 54, 28, 57, 86, 94, 18, 88, 17)
target_list = (33, 19, 63, 79, 46, 93, 50, 52, 8, 85)
target_list_2 = (49, 15, 66, 99, 98, 29, 74, 47, 58, 89)
cifar100_dm = cifar100_datamodule(labels=training_labels_2, data_dir=str(Path.home())+"/data")

cifar100_dm.prepare_data()

## Play with checkpoints

In [1]:
from models import DBModel
from pprint import pprint
import pytorch_lightning as pl
from Dropback import Dropback
import torch
from pl_bolts.datamodules import cifar10_datamodule

from torch.utils.tensorboard import SummaryWriter



checkpoint_path = "/data/sunxd/dropback_experiments/checkpoints/dropback_epoch348-val_accuracy0.83-val_loss0.77.ckpt"
checkpoint = torch.load(checkpoint_path)

checkpoint['optimizer_states'][0]['param_groups'][0]

# model = DBModel()

# # print(checkpoint['state_dict'])
# model.load_state_dict(checkpoint['state_dict'])
# # print(model.parameters())

# parameters=[]
# for name, param in model.named_parameters():
#     parameters.append(param.data.numpy())

# parameters
# cifar10_datamodule = cifar10_datamodule.CIFAR10DataModule()

# model = DBModel()

# optimizer = Dropback(
#             model.parameters(), 
#             lr=0.1, 
#             momentum=0.9, 
#             weight_decay=4e-5, 
#             track_size = 111835, 
#             init_decay = 0.1
#         )

# trainer = pl.Trainer(max_epochs = 0)



{'lr': 0.0012300000000000002,
 'momentum': 0.826,
 'dampening': 0,
 'weight_decay': 4e-05,
 'nesterov': False,
 'init_params': [tensor([[[[ 7.9788e-02, -8.8451e-02,  7.0427e-02],
            [ 3.1312e-02, -3.5135e-03, -9.4362e-02],
            [-1.1258e-02,  1.2473e-01, -5.7122e-02]],
  
           [[ 3.1116e-01, -4.7939e-03, -1.4347e-01],
            [-2.4974e-02, -1.6835e-01,  4.0705e-02],
            [-6.8739e-02,  6.4550e-02, -3.0489e-02]],
  
           [[ 7.2889e-02,  7.0259e-02, -3.6255e-02],
            [ 3.9132e-02, -1.5406e-01,  4.7702e-02],
            [-8.3027e-02, -4.2731e-02,  3.1306e-03]]],
  
  
          [[[ 2.2185e-01,  1.3868e-02,  2.5379e-02],
            [-1.6426e-01, -3.6051e-02,  1.9729e-02],
            [-1.5874e-01,  4.7021e-02,  6.5408e-02]],
  
           [[-9.3071e-02,  8.2292e-02, -1.3707e-02],
            [ 9.0022e-02, -9.5949e-02, -1.2578e-01],
            [-2.5947e-03, -7.4869e-02,  4.4828e-02]],
  
           [[-7.0264e-02, -5.3384e-02,  6.1359e-03],
  

In [ ]:
import torch
from pprint import pprint
from models import PruneModel

checkpoint_path = "/data/sunxd/dropback_experiments/checkpoints/prune-val_accuracy0.83-val_loss1.17_sparsity0.97.ckpt"

model = PruneModel(num_classes=10)
for name, module in model.named_modules():
    if hasattr(module, "weight") and module.weight is not None: 
        torch.nn.utils.prune.identity(module, "weight")
    if hasattr(module, "bias") and module.bias is not None: 
        torch.nn.utils.prune.identity(module, "bias")

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])  

for name, param in model.named_parameters():
    print(name, param)

for name, module in model.named_modules():
    if hasattr(module, "weight") and module.weight is not None: 
        torch.nn.utils.prune.remove(module, "weight")
    if hasattr(module, "bias") and module.bias is not None: 
        torch.nn.utils.prune.remove(module, "bias")

for name, param in model.named_parameters():
    print(name, param)